## 建構 Keras 模型：函數式模型

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
inputs = keras.Input(shape=(3,), name="my_input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)

model = keras.Model(inputs=inputs, outputs=outputs)

In [3]:
inputs.shape, inputs.dtype, features.shape

(TensorShape([None, 3]), tf.float32, TensorShape([None, 64]))

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 多輸入、多輸出的函數式模型

In [5]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

features = layers.Concatenate()([title, text_body, tags])
features = layers.Dense(64, activation="relu")(features)

priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
    num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 title (InputLayer)          [(None, 10000)]              0         []                            
                                                                                                  
 text_body (InputLayer)      [(None, 10000)]              0         []                            
                                                                                                  
 tags (InputLayer)           [(None, 100)]                0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 20100)                0         ['title[0][0]',               
                                                                     'text_body[0][0]',     

### 透過提供輸入及目標值陣列來訓練模型

In [7]:
import numpy as np

num_samples = 1280

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(optimizer="rmsprop",
       loss=["mean_squared_error", "categorical_crossentropy"],
       metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit([title_data, text_body_data, tags_data],
     [priority_data, department_data],
     epochs=1)

model.evaluate([title_data, text_body_data, tags_data],
        [priority_data, department_data])

priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])
priority_preds, department_preds

40/40 [==============================] - 0s 8ms/step


(array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], dtype=float32),
 array([[2.0511176e-03, 4.5276681e-07, 3.5154302e-09, 9.9794847e-01],
        [3.1792438e-03, 2.7878625e-06, 1.6836234e-08, 9.9681795e-01],
        [8.7982062e-03, 5.0037929e-06, 2.9986185e-09, 9.9119675e-01],
        ...,
        [2.5867200e-03, 1.5955457e-06, 3.3958913e-09, 9.9741173e-01],
        [4.3857940e-03, 1.3643615e-06, 4.3010506e-09, 9.9561286e-01],
        [7.6291645e-03, 1.5383766e-05, 7.5312325e-09, 9.9235547e-01]],
       dtype=float32))

### 透過提供輸入及目標值陣列的Python字典來訓練模型

In [8]:
model.compile(optimizer="rmsprop",
       loss={"priority": "mean_squared_error", "department": "categorical_crossentropy"},
       metrics={"priority": ["mean_absolute_error"], "department": ["accuracy"]})

model.fit(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data},
    {"priority": priority_data, "department": department_data},
     epochs=1)

model.evaluate(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data},
    {"priority": priority_data, "department": department_data})

priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

priority_preds, department_preds

40/40 [==============================] - 0s 9ms/step


(array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], dtype=float32),
 array([[3.4706996e-10, 9.9999934e-01, 3.6372616e-10, 6.1590600e-07],
        [1.7210604e-10, 9.9999982e-01, 1.6262879e-10, 9.5460720e-08],
        [1.6543196e-10, 9.9999982e-01, 1.3523239e-11, 7.8860062e-08],
        ...,
        [8.4206073e-11, 9.9999982e-01, 3.0939890e-11, 8.6780162e-08],
        [7.5373352e-10, 9.9999923e-01, 5.0148224e-10, 7.3534540e-07],
        [2.8212391e-11, 9.9999994e-01, 2.5561857e-12, 5.8853140e-09]],
       dtype=float32))

In [9]:
model.layers